## 查看训练集

In [7]:
from datasets import load_dataset
dataset1=load_dataset('csv',data_files='data/train.csv')

In [24]:
print(set(dataset1['train']['Fertilizer Name']))
name_to_id={k:v for v,k in enumerate(set(dataset1['train']['Fertilizer Name']))}
id_to_name=[v for v in set(dataset1['train']['Fertilizer Name'])]
print(id_to_name)
print(name_to_id)
print(dataset1)

{'DAP', '20-20', 'Urea', '14-35-14', '10-26-26', '28-28', '17-17-17'}
['DAP', '20-20', 'Urea', '14-35-14', '10-26-26', '28-28', '17-17-17']
{'DAP': 0, '20-20': 1, 'Urea': 2, '14-35-14': 3, '10-26-26': 4, '28-28': 5, '17-17-17': 6}
DatasetDict({
    train: Dataset({
        features: ['id', 'Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name'],
        num_rows: 750000
    })
})


In [22]:
id_to_name

['DAP', '20-20', 'Urea', '14-35-14', '10-26-26', '28-28', '17-17-17']

## 查看验证集

In [13]:
from datasets import load_dataset
dataset2=load_dataset('csv',data_files='data/test.csv')

Generating train split: 0 examples [00:00, ? examples/s]/home/adminad/zhangdw/workspace/uv/transformers/lib/python3.11/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 250000 examples [00:00, 863879.44 examples/s]


## 尝试构造prompt进行分类训练

In [1]:
def generate_fertilizer_prompt(
    temperature: float,
    humidity: float,
    moisture: float,
    soil_type: str,
    crop_type: str,
    nitrogen: float,
    potassium: float,
    phosphorous: float
) -> str:
    """
    生成用于预测肥料类型的prompt
    
    参数:
        temperature: 温度 (°C)
        humidity: 湿度 (%)
        moisture: 土壤湿度 (%)
        soil_type: 土壤类型 (如: Sandy, Loamy, Clayey)
        crop_type: 作物类型 (如: Wheat, Rice, Sugarcane)
        nitrogen: 土壤氮含量 (mg/kg)
        potassium: 土壤钾含量 (mg/kg)
        phosphorous: 土壤磷含量 (mg/kg)
    
    返回:
        格式化的prompt字符串
    """
    return f"""Predict the optimal fertilizer type for the given agricultural parameters:

{{
"Temperature": {temperature},  # Crop environment temperature in Celsius
"Humidity": {humidity},  # Relative humidity percentage
"Soil_Moisture": {moisture},  # Soil moisture content percentage
"Soil_Type": "{soil_type}",  # Soil texture (e.g., Sandy, Loamy, Clayey)
"Crop_Type": "{crop_type}",  # Type of crop (e.g., Wheat, Rice, Sugarcane)
"Nitrogen": {nitrogen},  # Soil nitrogen content in mg/kg
"Potassium": {potassium},  # Soil potassium content in mg/kg
"Phosphorus": {phosphorous}  # Soil phosphorus content in mg/kg
}}
"""

In [2]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,AutoConfig

/home/adminad/zhangdw/workspace/uv/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_path="/data/download-model/DeepSeek-R1-0528-Qwen3-8B"
num_labels=7

In [4]:
def load_model_and_tokenizer(model_path, num_labels):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    config = AutoConfig.from_pretrained(model_path)
    config.pad_token_id = tokenizer.pad_token_id
    config.num_labels = num_labels
    
    model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config)
    
    return model, tokenizer, config

In [5]:
model,tokenizer,_=load_model_and_tokenizer(model_path,num_labels)

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.01s/it]
Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at /data/download-model/DeepSeek-R1-0528-Qwen3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from datasets import load_dataset
dataset=load_dataset('csv',data_files='data/train.csv')

In [28]:
def tokenize_function(examples):
    examples["text"] = [
        generate_fertilizer_prompt(
            temp, hum, mois, soil, crop, nit, pot, phos
        )
        for temp, hum, mois, soil, crop, nit, pot, phos in zip(
            examples["Temparature"], examples["Humidity"], examples["Moisture"],
            examples["Soil Type"], examples["Crop Type"], examples["Nitrogen"],
            examples["Potassium"], examples["Phosphorous"]
        )
    ]
    import json
    with open('mapping.json','r') as f:
        mapping=json.load(f)
        name_to_id=mapping['name_to_id']
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    
    tokenized['labels']=[name_to_id[cat] for cat in examples['Fertilizer Name']]
    
    return tokenized

In [29]:
dataset = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 750000/750000 [03:30<00:00, 3555.89 examples/s]


In [ ]:
max_len=0
max_id=0
for i in range(len(dataset['train']['text'])):
    if max_len<len(dataset['train']['text'][i]):
        max_len=len(dataset['train']['text'][i])
        max_id=i
print(max_len,max_id)
print(dataset['train']['text'][max_id])

## 推理时拿到top 3的结果